In [1]:
#def train_revenue_prediction_model(session: Session, features_table: str,

In [7]:
# Snowpark for Python
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import IntegerType, StringType, StructType, FloatType, StructField, DateType, Variant
from snowflake.snowpark.functions import udf, sum, col,array_construct,month,year,call_udf,lit
from snowflake.snowpark.version import VERSION
# Misc
import json
import pandas as pd

In [8]:
# Create Snowflake Session object
connection_parameters = json.load(open('Advertising-Spend-ROI-Prediction/connection.json'))
session = Session.builder.configs(connection_parameters).create()

snowflake_environment = session.sql('select current_user(), current_role(), current_database(), current_schema(), current_version(), current_warehouse()').collect()
snowpark_version = VERSION

# Current Environment Details
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(snowflake_environment[0][1]))
print('Database                    : {}'.format(snowflake_environment[0][2]))
print('Schema                      : {}'.format(snowflake_environment[0][3]))
print('Warehouse                   : {}'.format(snowflake_environment[0][5]))
print('Snowflake version           : {}'.format(snowflake_environment[0][4]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

User                        : GUSCONTINUAL
Role                        : ACCOUNTADMIN
Database                    : DWH
Schema                      : PUBLIC
Warehouse                   : COMPUTE_WH
Snowflake version           : 6.30.0
Snowpark for Python version : 0.10.0


In [15]:
def snow_quantile(session: Session, features_table: str, out_table: str) -> Variant:
    df = session.table(features_table).to_pandas()
    quantile_list = [0, .25, .5, .75, 1.]
    quantiles = df['TOTAL_TIME_SPENT_ON_WEBSITE'].quantile(quantile_list)
    df['TIME_ON_WEBSITE_QUANTILE_RANGE'] = pd.qcut(
                                            df['TOTAL_TIME_SPENT_ON_WEBSITE'], 
                                            q=quantile_list)
    quantile_labels = ['1Q', '2Q', '3Q', '4Q']
    df['TIME_ON_WEBSITE_QUANTILE_LABELS'] = pd.qcut(
                                            df['TOTAL_TIME_SPENT_ON_WEBSITE'], 
                                            q=quantile_list,       
                                            labels=quantile_labels)
    df = df.drop('TIME_ON_WEBSITE_QUANTILE_RANGE', axis=1)
    foo_tbl = session.create_dataframe(df)
    foo_tbl.write.mode('overwrite').save_as_table(out_table)
    
    return str(df[['TOTAL_TIME_SPENT_ON_WEBSITE','TIME_ON_WEBSITE_QUANTILE_LABELS']].head())

In [16]:
session.sproc.register(func=snow_quantile,
                       name="quantile_df_test",
                       packages=['snowflake-snowpark-python','pandas'],
                       is_permanent=True,
                       stage_location="@dash_sprocs",
                       replace=True)

In [17]:
print(session.call('quantile_df_test','LEAD_SCORING_Q', 'LEAD_SCORING_K'))

"   TOTAL_TIME_SPENT_ON_WEBSITE TIME_ON_WEBSITE_QUANTILE_LABELS\n0                            0                              1Q\n1                          674                              3Q\n2                         1532                              4Q\n3                          305                              3Q\n4                         1428                              4Q"
